In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ml-finalexam-p3/train.csv
/kaggle/input/2022-ml-finalexam-p3/test.csv
/kaggle/input/2022-ml-finalexam-p3/sample_submit.csv


In [2]:
# data를 가져온다

train = pd.read_csv('../input/2022-ml-finalexam-p3/train.csv')
test = pd.read_csv('../input/2022-ml-finalexam-p3/test.csv')
submit = pd.read_csv('../input/2022-ml-finalexam-p3/sample_submit.csv')

In [3]:
trainX = train.drop(['Unnamed: 0', '일자', '조식메뉴', '중식메뉴', '석식메뉴','중식계'], axis=1)
trainY = train['중식계']
testX = test.drop(['Unnamed: 0', '조식메뉴', '중식메뉴', '석식메뉴', '일자'], axis=1)

In [4]:
# 라벨인코더를 통해 실수화한다.

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

trainX['요일'] = le.fit_transform(trainX['요일'])
testX['요일'] = le.transform(testX['요일'])

In [5]:
# 각각의 배열에 넣는다

breakfast_train = [x for x in train['조식메뉴']]
lunch_train = [x for x in train['중식메뉴']]
dinner_train = [x for x in train['석식메뉴']]

breakfast_test = [x for x in test['조식메뉴']]
lunch_test = [x for x in test['중식메뉴']]
dinner_test = [x for x in test['석식메뉴']]

In [6]:
# CountVectorizer를 이용하여 text를 분류한다.

from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()

vec.fit(breakfast_train + breakfast_test + lunch_train+ lunch_test + dinner_train + dinner_test)

breakfast_train = vec.transform(breakfast_train).toarray()
breakfast_test = vec.transform(breakfast_test).toarray()

lunch_train = vec.transform(lunch_train).toarray()
lunch_test = vec.transform(lunch_test).toarray()

dinner_train = vec.transform(dinner_train).toarray()
dinner_test = vec.transform(dinner_test).toarray()

In [7]:
x_train = pd.concat([trainX, pd.DataFrame(breakfast_train), pd.DataFrame(lunch_train), pd.DataFrame(dinner_train)], axis=1)
x_test = pd.concat([testX, pd.DataFrame(breakfast_test), pd.DataFrame(lunch_test), pd.DataFrame(dinner_test)], axis=1)

In [8]:
# 회귀문제로 RandomForestRegressor를 사용하였다.
# 그 다음 모델로 train 데이터를 학습 시켜준다.

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=1)
model.fit(x_train, trainY)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


RandomForestRegressor(random_state=1)

In [9]:
pred = model.predict(x_test)
submit['중식계'] = pred

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [10]:
submit.to_csv('result4.csv', index=False)